In [1]:
from init import *
import json
from pyeoskit import eosapi
from evm import get_eth_address_info, w3

#eosapi.set_nodes(['https://nodes.uuos.network:8443'])
eosapi.set_nodes(['http://127.0.0.1:8888'])
#eosapi.set_nodes(['http://testapi.uuos.network'])


config.main_token='EOS'
wallet.import_key('test', '5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc')

priv_keys = [
'5KQwrPbwdL6PhXujxW37FSSQZ1JiwsST4cqQzDeyXtP79zkvFD3',#EOS6MRyAjQq8ud7hVNYcfnVPJqcVpscN5So8BhtHuGYqET5GDW5CV
'5JEcwbckBCdmji5j8ZoMHLEUS8TqQiqBG1DRx1X9DN124GUok9s',#EOS61MgZLN7Frbc2J7giU7JdYjy2TqnfWFjZuLXvpHJoKzWAj7Nst
'5JbDP55GXN7MLcNYKCnJtfKi9aD2HvHAdY7g8m67zFTAFkY1uBB',#EOS5JuNfuZPATy8oPz9KMZV2asKf9m8fb2bSzftvhW55FKQFakzFL
'5K463ynhZoCDDa4RDcr63cUwWLTnKqmdcoTKTHBjqoKfv4u5V7p',#EOS8Znrtgwt8TfpmbVpTKvA2oB8Nqey625CLN8bCN3TEbgx86Dsvr
'5KH8vwQkP4QoTwgBtCV5ZYhKmv8mx56WeNrw9AZuhNRXTrPzgYc',#EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV
'5KT26sGXAywAeUSrQjaRiX9uk9uDGNqC1CSojKByLMp7KRp8Ncw',#EOS8Ep2idd8FkvapNfgUwFCjHBG4EVNAjfUsRRqeghvq9E91tkDaj
]
for priv_key in priv_keys:
    wallet.import_key('test', priv_key)

db.reset()
print('done!')

done!


# Add eosio.code to active permission

In [2]:
account_name = 'helloworld11'
a = {
    "account": account_name,
    "permission": "active",
    "parent": "owner",
    "auth": {
        "threshold": 1,
        "keys": [
            {
                "key": "EOS7ent7keWbVgvptfYaMYeF2cenMBiwYKcwEuc11uCbStsFKsrmV",
                "weight": 1
            },
        ],
        "accounts": [{"permission":{"actor":account_name,"permission":"eosio.code"},"weight":1}],
        "waits": []
    }
}
r = eosapi.push_action('eosio', 'updateauth', a, {account_name:'owner'})
print('done!')

done!


# Deploy evm smart contract to network

In [3]:
eosapi.clear_abi_cache('helloworld11');
source_path = 'ethereum_vm'
evm_code_file = os.path.join(source_path, 'ethereum_vm.cpp')
deploy_cpp_contract_from_file('helloworld11', evm_code_file, includes = [source_path])
print('Done!')

Done!


# Create an ETH account and bind it to an EOS account

In [48]:
main_account = 'helloworld11'
test_account = 'helloworld12'
eth_address = None
name = 'helloworld11'
args = {'account':'helloworld12', 'text': 'hello,world'}
try:
    r = eosapi.push_action(name, 'create', args, {'helloworld12':'active'})
    print('++++console:', r['processed']['action_traces'][0]['console'])
    eth_address = r['processed']['action_traces'][0]['receipt']['return_value']
    print('+++', eth_address)
    eth_address = w3.toChecksumAddress(eth_address)
    print('eth address:', eth_address)
    print(r['processed']['elapsed'])
except Exception as e:
    parsed = json.loads(e.response)
    print('+++error:\n', json.dumps(parsed, indent=4))

error:
 {
    "code": 500,
    "message": "Internal Service Error",
    "error": {
        "code": 3050003,
        "name": "eosio_assert_message_exception",
        "what": "eosio_assert_message assertion failure",
        "details": [
            {
                "message": "assertion failure with message: eth address already bind to an EOS account",
                "file": "wasm_interface.cpp",
                "line_number": 1091,
                "method": "eosio_assert"
            },
            {
                "message": "pending console output: f375c0d7e20a15b3d35af771763ca166d54cce97\n",
                "file": "apply_context.cpp",
                "line_number": 121,
                "method": "exec_one"
            }
        ]
    }
}


### Verify eth address method 1

In [52]:
import rlp
import hashlib
from eth_utils import keccak
e = rlp.encode(['helloworld12', 'hello,world'])
h = keccak(e)
if not eth_address:
    assert eth_address == h[12:].hex()
else:
    eth_address = h[12:].hex()
    print(eth_address)

f375c0d7e20a15b3d35af771763ca166d54cce97


### Verify result method 2

In [30]:
#install pysah3
%system python3.7 -m pip install pysha3

['Requirement already satisfied: pysha3 in /Users/newworld/opt/anaconda3/lib/python3.7/site-packages (1.0.2)']

In [54]:
import rlp
import sha3
e = rlp.encode(['helloworld12', 'hello,world'])
h = sha3.keccak_256()
h.update(e)
h = h.digest()
if eth_address:
    assert eth_address == h[12:].hex()
else:
    eth_address = h[12:].hex()
    print(eth_address)

In [56]:
eth_address = w3.toChecksumAddress(eth_address)
eth_address

'0xf375C0D7E20a15b3D35AF771763CA166D54cce97'

# Deposit Test

In [5]:
r = eosapi.transfer('helloworld12', 'helloworld11', 10.0, 'deposit')

In [8]:
get_eth_address_info('helloworld11', eth_address)

{'nonce': 9, 'balance': '77.0000 SYS'}

In [216]:
r = eosapi.transfer('helloworld12', 'helloworld11', 1.0, 'deposit')

In [217]:
get_eth_address_info('helloworld11', eth_address)

+++args: 0100000000000000b0ad0100000000000453595300000000


b'{"nonce":1,"balance":"11.0000 SYS"}'

# Withdraw Test

In [218]:
eosapi.get_balance('helloworld12')

9999989.0

### Check balance in ETH account

In [219]:
get_eth_address_info('helloworld11', eth_address)

+++args: 0100000000000000b0ad0100000000000453595300000000


b'{"nonce":1,"balance":"11.0000 SYS"}'

### Withdraw 1.0000

In [220]:
name = 'helloworld12'
args = {'account': name, 'amount': '1.0000 SYS'}
try:
    r = eosapi.push_action('helloworld11', 'withdraw', args, {name:'active'})
    print('++++console:', r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)

++++console: 4,SYS4,SYS
836


### Check balance

In [224]:
eosapi.get_balance('helloworld12')

9999990.0

In [171]:
eth_address

'0xf375C0D7E20a15b3D35AF771763CA166D54cce97'

### Check balance in binded ETH account

In [4]:
get_eth_address_info('helloworld11', eth_address)

b'{"nonce":1,"balance":"10.0000 SYS"}'

bind an eth address through directly
```python
name = 'helloworld11'
args = bytes.fromhex('b654A7A81E0aeb7721A22F27A04ecf5aF0E8a9A3')
try:
    r = eosapi.push_action(name, 'activate', args, {name:'active'})
    print('++++console:', r['processed']['action_traces'][0]['console'])
    print(r['processed']['elapsed'])
except Exception as e:
    print(e)
```

# Deploy an ethereum smart contract

In [5]:
from solcx import compile_source, compile_files

greeter = '''pragma solidity ^0.6.0;
contract Greeter {
    uint value;
    event onSetValue(uint value);
    event onGetValue(uint value);
    constructor() public {
        value = 1;
    }

    function getValue2() payable public returns (uint){
//        msg.sender.transfer(1000);
        emit onGetValue(value);
        return value;
    }

    function getValue() payable public returns (string memory){
        return "hello,world";
    }

    function setValue(uint v) payable public {
        value = v;
        emit onSetValue(v);
    }
}
'''

def compile(contract_source_code, main_class):
    compiled_sol = compile_source(contract_source_code) # Compiled source code
    contract_interface = compiled_sol[main_class]

    return contract_interface


main_class = '<stdin>:Greeter'
contract_source_code = greeter
contract_interface = compile(contract_source_code, main_class)
bytecode = contract_interface['bin']
abi = contract_interface['abi']
print('Done!')

Done!


In [6]:
Greeter = w3.eth.contract(abi=abi, bytecode=bytecode)
ret = Greeter.constructor().transact({'from':'0xf375C0D7E20a15b3D35AF771763CA166D54cce97'})
logs = ret['processed']['action_traces'][0]['console']
print('logs:', logs)
output = ret['processed']['action_traces'][0]['receipt']['return_value']
output = bytes.fromhex(output)
output = output.decode('utf8')
print('+++++output:', output)
output = json.loads(output)
print(output)
contract_address = w3.toChecksumAddress(output['new_address'])
print(contract_address)

----request_func eth_sendTransaction [{'from': '0xf375C0D7E20a15b3D35AF771763CA166D54cce97', 'data': '0x608060405234801561001057600080fd5b5060016000819055506101ff806100286000396000f3fe6080604052600436106100345760003560e01c8063209652551461003957806355241077146100bc578063c515205d146100ea575b600080fd5b610041610108565b6040518080602001828103825283818151815260200191508051906020019080838360005b83811015610081578082015181840152602081019050610066565b50505050905090810190601f1680156100ae5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b6100e8600480360360208110156100d257600080fd5b8101908080359060200190929190505050610145565b005b6100f2610186565b6040518082815260200191505060405180910390f35b60606040518060400160405280600b81526020017f68656c6c6f2c776f726c64000000000000000000000000000000000000000000815250905090565b806000819055507ff48a1dc57eefa3de4406e65fc0fc377b25d100bd06429201a01a9e8e0e1d0773816040518082815260200191505060405180910390a150565b60007f29325e60ff39fad605f9bb4f2fd7

#### Call setValue in ethereum smart contract

In [7]:
contract_address

'0x0633A42c777f64f895dE1B0097de00C8D181A5e9'

In [15]:
args = {'from': eth_address,'to':contract_address}
ret = Greeter.functions.setValue(0xffaabbcc).transact(args)
logs = ret['processed']['action_traces'][0]['console']
output = ret['processed']['action_traces'][0]['receipt']['return_value']
output = bytes.fromhex(output)
output = output.decode('utf8')
output = json.loads(output)
print('+++logs:', logs)
print('+++output:', output)

----request_func eth_sendTransaction [{'from': '0xf375C0D7E20a15b3D35AF771763CA166D54cce97', 'to': '0x0633A42c777f64f895dE1B0097de00C8D181A5e9', 'data': '0x5524107700000000000000000000000000000000000000000000000000000000ffaabbcc', 'gas': 100088}]
{'to': '0x0633A42c777f64f895dE1B0097de00C8D181A5e9', 'data': '0x5524107700000000000000000000000000000000000000000000000000000000ffaabbcc', 'gas': 20000000, 'nonce': 0, 'gasPrice': 1}
++++ HTTPConnectionPool(host='198.54.123.58', port=8800): Max retries exceeded with url: /v1/chain/get_info (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x108526610>: Failed to establish a new connection: [Errno 61] Connection refused'))


TypeError: 'NoneType' object is not subscriptable

In [208]:
a = '[{"address":"3465938531e1bd44d4ce10fb849b47ba7fed073f","data" :"5524107700000000000000000000000000000000000000000000000000000000ffaabbcc","topics" :["f48a1dc57eefa3de4406e65fc0fc377b25d100bd06429201a01a9e8e0e1d0773"]}]'
aa = json.loads(a)
print(aa)

[{'address': '3465938531e1bd44d4ce10fb849b47ba7fed073f', 'data': '5524107700000000000000000000000000000000000000000000000000000000ffaabbcc', 'topics': ['f48a1dc57eefa3de4406e65fc0fc377b25d100bd06429201a01a9e8e0e1d0773']}]


# transfer test

In [10]:
get_eth_address_info('helloworld11', eth_address)

b'{"nonce":4,"balance":"10.0000 SYS"}'

In [11]:
get_eth_address_info('helloworld11', contract_address)

b'{"nonce":2,"balance":"0.0000 SYS"}'

### setValue and transfer 1.0 to contract address

In [12]:
print('transfer from ', eth_address, 'to', contract_address)
args = {'from': eth_address,'to':contract_address, 'value':10000}
ret = Greeter.functions.setValue(0xffaabbcc).transact(args)
logs = ret['processed']['action_traces'][0]['console']
output = ret['processed']['action_traces'][0]['receipt']['return_value']
output = bytes.fromhex(output)
output = output.decode('utf8')
output = json.loads(output)
print(logs)
print(output)

transfer from  0xf375C0D7E20a15b3D35AF771763CA166D54cce97 to 0x0633A42c777f64f895dE1B0097de00C8D181A5e9
----request_func eth_sendTransaction [{'from': '0xf375C0D7E20a15b3D35AF771763CA166D54cce97', 'to': '0x0633A42c777f64f895dE1B0097de00C8D181A5e9', 'value': 10000, 'data': '0x5524107700000000000000000000000000000000000000000000000000000000ffaabbcc', 'gas': 100088}]
{'to': '0x0633A42c777f64f895dE1B0097de00C8D181A5e9', 'value': 10000, 'data': '0x5524107700000000000000000000000000000000000000000000000000000000ffaabbcc', 'gas': 20000000, 'nonce': 0, 'gasPrice': 1}
[{"address":"0633a42c777f64f895de1b0097de00c8d181a5e9","data" :"5524107700000000000000000000000000000000000000000000000000000000ffaabbcc","topics" :["f48a1dc57eefa3de4406e65fc0fc377b25d100bd06429201a01a9e8e0e1d0773"]}]
{'new_address': '0000000000000000000000000000000000000000', 'output': ''}


### get address info

In [13]:
get_eth_address_info('helloworld11', eth_address)

b'{"nonce":5,"balance":"9.0000 SYS"}'

In [14]:
get_eth_address_info('helloworld11', contract_address)

b'{"nonce":2,"balance":"1.0000 SYS"}'

In [119]:
w3.keccak(text="onSetValue(uint256)").hex()

'0xf48a1dc57eefa3de4406e65fc0fc377b25d100bd06429201a01a9e8e0e1d0773'

In [13]:
a = w3.eth.account.privateKeyToAccount('0x2a2a401e99b8b032fcb20c320af2bc066222eba7c0496e012200e58caf1bfb5a')
print(a.key, a.address)

b'**@\x1e\x99\xb8\xb02\xfc\xb2\x0c2\n\xf2\xbc\x06b"\xeb\xa7\xc0In\x01"\x00\xe5\x8c\xaf\x1b\xfbZ' 0xb654A7A81E0aeb7721A22F27A04ecf5aF0E8a9A3


In [ ]:
from web3.auto import w3
acct = w3.eth.account.create('KEYSMASH FJAFJKLDSKF7JKFDJ 1530')
print(acct.address)
acct.privateKey.hex()
#0xb654A7A81E0aeb7721A22F27A04ecf5aF0E8a9A3
#'0x2a2a401e99b8b032fcb20c320af2bc066222eba7c0496e012200e58caf1bfb5a'